### Task II: Classical Graph Neural Network (GNN)

For Task II, you will use ParticleNet’s data for Quark/Gluon jet classification available here with its corresponding description. 
* Choose 2 Graph-based architectures of your choice to classify jets as being quarks or gluons. Provide a description on what considerations you have taken to project this point-cloud dataset to a set of interconnected nodes and edges. 
* Discuss the resulting performance of the 2 chosen architectures. 


In [2]:
!pip install energyflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 700.5/700.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.2/502.2 kB 36.2 MB/s eta 0:00:00


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tf_keras_model import get_particle_net_lite
import energyflow
from tensorflow import keras

In [4]:
qg_dataset = energyflow.qg_jets.load(num_data=100000, pad=True, ncol=4, generator='pythia',with_bc=False, cache_dir='~/.energyflow')

URL fetch failure on https://www.dropbox.com/s/fclsl7pukcpobsb/QG_jets.npz?dl=1: None -- Not Found
Failed to download QG_jets.npz from source 'dropbox', trying next source...


In [5]:
x = qg_dataset[0]

print("Shape of X: ", x.shape)
print("X: pt, rapidity, azimuthal angle, and pdgid ",x[0][0])

Shape of X:  (100000, 139, 4)
X: pt, rapidity, azimuthal angle, and pdgid  [ 0.26876914  0.35690317  4.74138734 22.        ]


In [6]:
y = qg_dataset[1]
print("y: 0 or 1 (gluon or quark): ",y[0])
print("Shape of y:", y.shape)
y = keras.utils.to_categorical(y)
print("y after one-Hot Encoding: [quark, gluon]  ", y[0])
print("Shape of y:", y.shape)

y: 0 or 1 (gluon or quark):  1.0
Shape of y: (100000,)
y after one-Hot Encoding: [quark, gluon]   [0. 1.]
Shape of y: (100000, 2)


## Data Pre-processing
* Train Test Validation Split
* Masking
* 3-dimensional feature datasets

In [7]:
# Shuffling the dataset to avoiad any unpecedented bias
x, y = shuffle(x, y, random_state=0)

# Splitting the dataset into train (70% of the datapoints) and test (30% of the datapoints)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

# Further, splitting the test into 50% test and 50% validation dataset
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

print("Shape of training dataset :", x_train.shape, y_train.shape)
print("Shape of testing dataset :", (x_test.shape, y_test.shape))
print("Shape of validation dataset :", (x_val.shape, y_val.shape))

Shape of training dataset : (70000, 139, 4) (70000, 2)
Shape of testing dataset : ((15000, 139, 4), (15000, 2))
Shape of validation dataset : ((15000, 139, 4), (15000, 2))


In [8]:
# Masking

# This function call computes the sum of x_train along its third axis (axis=2).

mask_train = np.sum(x_train, axis=2)
print(f"Before: The shape of x_train is {x_train.shape} where {x_train.shape[0]} is the number of samples, {x_train.shape[1]} is the number of features, and {x_train.shape[2]} is the number of components in each feature vector")
print(f"After: The result will have a shape of {mask_train.shape}, where each element now represents the sum of the components for a given feature in the sample.\n")

print(f"Instance of Masked X:\n {mask_train[0]}")

# Binary Mask

# Identifying Non-Zero Features:

mask_train = np.array(mask_train != 0, np.float32)

# Reshaping mask for a third axis
mask_train = mask_train.reshape(x_train.shape[0], x_train.shape[1], 1)
print(f"Binary Mask:\n {mask_train[0]}")

Before: The shape of x_train is (70000, 139, 4) where 70000 is the number of samples, 139 is the number of features, and 4 is the number of components in each feature vector
After: The result will have a shape of (70000, 139), where each element now represents the sum of the components for a given feature in the sample.

Instance of Masked X:
 [   29.20243269    27.50561143    30.26089051   220.41819997
    27.95042087    28.15969933   329.0687446     27.57628735
  -206.06003465   218.08468374    26.41602355    27.02195013
    31.79622486    26.59246689    26.88924931  -202.09822736
    26.71899363    31.23462352  -310.42681747    27.70451205
    30.7169403   -198.56554041  2233.77838649    36.42720615
    29.23621805  -187.38676832    33.46071738   268.17256202
    32.3521241  -2054.01856237   118.7019814     56.63953271
   151.53934524    57.07618763     0.             0.
     0.             0.             0.             0.
     0.             0.             0.             0.
     0.

In [9]:
# Similarly for Test and Validation dataset
mask_val = np.sum(x_val, axis=2)
mask_val = np.array(mask_val != 0, np.float32)
mask_val = mask_val.reshape(x_val.shape[0], x_val.shape[1], 1)

mask_test = np.sum(x_test, axis=2)
mask_test = np.array(mask_test != 0, np.float32)
mask_test = mask_val.reshape(x_test.shape[0], x_test.shape[1], 1)


In [10]:
train_dataset = {
    'points': x_train[:, :, 1:3],
    'features': x_train,
    'mask': mask_train
}

test_dataset = {
    'points': x_test[:, :, 1:3],
    'features': x_test,
    'mask': mask_test
}

val_dataset = {
    'points': x_val[:, :, 1:3],
    'features': x_val,
    'mask': mask_val
}

In [11]:
num_classes = 2
shapes = {k:train_dataset[k].shape[1:] for k in train_dataset}
# shapes: {'points': (139, 2), 'features': (139, 4), 'mask': (139, 1)}

model = get_particle_net_lite(num_classes, shapes)

# Training parameters
batch_size = 1024
epochs = 100

# Learning Rate
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    print('Learning rate: %f' % lr)
    return lr

# Compiling Model
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.legacy.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy',keras.metrics.AUC()])

# Model Summary
model.summary()

Learning rate: 0.001000
Model: "ParticleNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 mask (InputLayer)           [(None, 139, 1)]             0         []                            
                                                                                                  
 tf.math.not_equal (TFOpLam  (None, 139, 1)               0         ['mask[0][0]']                
 bda)                                                                                             
                                                                                                  
 tf.cast (TFOpLambda)        (None, 139, 1)               0         ['tf.math.not_equal[0][0]']   
                                                                                                  
 tf.math.equal (TFOpLambda)  (None, 139, 1)               0     

In [12]:
# Setting the callbacks
checkpoint_path_1 = "GNN_checkpoints/cp.ckpt"
checkpoint_dir_1 = os.path.dirname(checkpoint_path_1)

cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path_1, verbose=0, save_weights_only=True)
lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [lr_scheduler, progress_bar, cp_callback]

# Fitting the model on training dataset
history = model.fit(train_dataset, y_train,
                          batch_size=batch_size,
                          epochs=epochs,
                          validation_data=(val_dataset, y_val),
                          shuffle=True,
                          callbacks=callbacks)

Learning rate: 0.001000
Epoch 1/100
69/69 [==============================] - 63s 912ms/sample - loss: 0.5261 - accuracy: 0.7381 - auc: 0.8173 - val_loss: 0.5808 - val_accuracy: 0.7055 - val_auc: 0.7925 - lr: 0.0010
Learning rate: 0.001000
Epoch 2/100
69/69 [==============================] - 53s 763ms/sample - loss: 0.4828 - accuracy: 0.7744 - auc: 0.8497 - val_loss: 0.5238 - val_accuracy: 0.7718 - val_auc: 0.8502 - lr: 0.0010
Learning rate: 0.001000
Epoch 3/100
69/69 [==============================] - 52s 756ms/sample - loss: 0.4772 - accuracy: 0.7783 - auc: 0.8536 - val_loss: 0.4943 - val_accuracy: 0.7831 - val_auc: 0.8576 - lr: 0.0010
Learning rate: 0.001000
Epoch 4/100
69/69 [==============================] - 49s 710ms/sample - loss: 0.4713 - accuracy: 0.7833 - auc: 0.8577 - val_loss: 0.4787 - val_accuracy: 0.7835 - val_auc: 0.8583 - lr: 0.0010
Learning rate: 0.001000
Epoch 5/100
69/69 [==============================] - 52s 755ms/sample - loss: 0.4643 - accuracy: 0.7885 - auc: 0.862

In [15]:
loss, acc, auc = model.evaluate(test_dataset, y_test)
print("Evaluation Metric of model on test data")
print("Loss:", loss)
print("Accuracy:", acc)
print("AUC:", auc)

469/469 [==============================] - 9s 19ms/step - loss: 0.5138 - accuracy: 0.7533 - auc: 0.8271
Evaluation Metric of model on test data
Loss: 0.513751745223999
Accuracy: 0.753333330154419
AUC: 0.8270620703697205


# Why this approach?
This approach enables the model to learn both local and global interactions within the jet. Using graph-based deep learning for quark and gluon jet classification is a well-founded approach, as it aligns naturally with the underlying structure of jet events. Rather than treating jets as images or fixed-dimensional arrays, Graph Neural Networks (GNNs) provide a more flexible and expressive way to capture relationships between constituent particles.<br>
<br>
<br>
Graph Neural Networks (GNNs) and ParticleNet are both models for graph-based tasks, but they differ in their applications and designs.
<br>
**GNN because**: Permutation Invariance; The model remains unaffected by the ordering of particles. GNNs are a general framework for tasks involving graphs, where nodes and edges represent entities and their relationships, respectively. They use a message-passing mechanism to learn node or graph-level representations.
Dynamic Edge Construction; Learns particle relationships beyond fixed spatial structures.
Efficient Feature Learning; Message-passing mechanisms enhance representation learning.
<br><br>
**ParticleNet because**, a state-of-the-art GNN for jet tagging, employs EdgeConv layers to dynamically learn particle interactions, outperforming traditional CNNs in jet classification. ParticleNet, on the other hand, is a specialized GNN for particle physics tasks, such as jet classification and particle tracking. It uses particle-specific features, such as Lorentz invariant quantities, and is tailored to capture interactions between particles in high-energy physics events. While GNNs have broad applications across fields, ParticleNet is specifically built to handle the unique characteristics of particle data.
<br>
We are evaluating two GNN models because : 
A broader understanding of different graph-based approaches.<br>
A fair assessment of performance trade-offs (accuracy, efficiency, generalization).<br>
Insights into optimal jet representation for classification.<br>

Although there is scope of improvement as we can see its performance, optimisations in model architecture enhancements, data respresentations staregies, hypermeter optimisation,etc.